## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")

In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Carauari,BR,2021-12-04 17:55:02,-4.8828,-66.8958,90.59,53,73,2.62,broken clouds
1,1,Vaini,TO,2021-12-04 17:55:03,-21.2000,-175.2000,78.96,94,40,8.05,moderate rain
2,2,Avarua,CK,2021-12-04 17:55:03,-21.2078,-159.7750,75.25,94,100,3.44,overcast clouds
10,10,Rikitea,PF,2021-12-04 17:55:07,-23.1203,-134.9692,75.54,73,60,12.39,broken clouds
17,17,Botucatu,BR,2021-12-04 17:55:09,-22.8858,-48.4450,84.87,33,12,1.97,few clouds
18,18,Kudahuvadhoo,MV,2021-12-04 17:55:10,2.6708,72.8944,82.47,77,24,12.50,few clouds
21,21,Umm Kaddadah,SD,2021-12-04 17:55:11,13.6017,26.6876,79.65,20,0,10.54,clear sky
24,24,Butaritari,KI,2021-12-04 17:55:13,3.0707,172.7902,82.62,77,33,16.35,scattered clouds
25,25,Karratha,AU,2021-12-04 17:55:13,-20.7377,116.8463,88.12,29,20,15.73,few clouds
31,31,Isangel,VU,2021-12-04 17:55:15,-19.5500,169.2667,77.59,85,92,9.51,overcast clouds


In [5]:
preferred_cities_df.count()

City_ID                212
City                   212
Country                211
Date                   212
Lat                    212
Lng                    212
Max Temp               212
Humidity               212
Cloudiness             212
Wind Speed             212
Current Description    212
dtype: int64

In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                1
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Date                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carauari,BR,90.59,broken clouds,-4.8828,-66.8958,
1,Vaini,TO,78.96,moderate rain,-21.2000,-175.2000,
2,Avarua,CK,75.25,overcast clouds,-21.2078,-159.7750,
10,Rikitea,PF,75.54,broken clouds,-23.1203,-134.9692,
17,Botucatu,BR,84.87,few clouds,-22.8858,-48.4450,
18,Kudahuvadhoo,MV,82.47,few clouds,2.6708,72.8944,
21,Umm Kaddadah,SD,79.65,clear sky,13.6017,26.6876,
24,Butaritari,KI,82.62,scattered clouds,3.0707,172.7902,
25,Karratha,AU,88.12,few clouds,-20.7377,116.8463,
31,Isangel,VU,77.59,overcast clouds,-19.5500,169.2667,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print(hotels["results"][0]["name"])
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel Fellip
Keleti Beach Resort
Paradise Inn
People ThankYou
Primar Plaza Hotel
Niyama Private Islands Maldives
Hotel not found... skipping.
Isles Sunset Lodge
ibis Styles Karratha
Tanna Lodge
Shandrani Beachcomber Resort & Spa
CHACARA BAILLY
Hotel Ezequiel
"Plantation Bed and Breakfast
Hotel Xaguate
Hotel not found... skipping.
Oceano
Belchior Palace Hotel
Hotel Maria
Hotels in Lakshadweep Islands
Raybow International Hotel
Waterfall Hostel
Lorengau Harbourside Hotel
Casa Dora, Bubaque
Mercure Padang
Motel 6 Sinton, TX
asi es
Maison d'hôtes EBEN-EZER DE BOUSSE
Uruna Bay Retreat
Storyboard Beach Resort
Villa Enata
Hotel Santika Luwuk
Hotel not found... skipping.
Casa Tia Villa
Hotel Praia Morena
Cititel Penang
Pousada Porto Praia
Hotel Kou-Bugny
ibis Balikpapan
Villa Veronica Resort Private Pool
Quality Suites Vila Velha
Hotel not found... skipping.
LIKWALE LODGE
Vahmoneh Lodge Vanimo
Hotel Citra
Desert Gardens Hotel - Ayers Rock Resort
Pension Oaoa
Hotel not found... skipping.
Hotel 

In [10]:
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carauari,BR,90.59,broken clouds,-4.8828,-66.8958,Hotel Fellip
1,Vaini,TO,78.96,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
2,Avarua,CK,75.25,overcast clouds,-21.2078,-159.7750,Paradise Inn
10,Rikitea,PF,75.54,broken clouds,-23.1203,-134.9692,People ThankYou
17,Botucatu,BR,84.87,few clouds,-22.8858,-48.4450,Primar Plaza Hotel


In [11]:
hotel_df.replace("", np.nan, inplace=True)

In [12]:
hotel_df.count()

City                   211
Country                211
Max Temp               211
Current Description    211
Lat                    211
Lng                    211
Hotel Name             198
dtype: int64

In [13]:
clean_hotel_df = hotel_df.dropna()

In [14]:
clean_hotel_df.count()

City                   198
Country                198
Max Temp               198
Current Description    198
Lat                    198
Lng                    198
Hotel Name             198
dtype: int64

In [15]:
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Carauari,BR,90.59,broken clouds,-4.8828,-66.8958,Hotel Fellip
1,Vaini,TO,78.96,moderate rain,-21.2000,-175.2000,Keleti Beach Resort
2,Avarua,CK,75.25,overcast clouds,-21.2078,-159.7750,Paradise Inn
10,Rikitea,PF,75.54,broken clouds,-23.1203,-134.9692,People ThankYou
17,Botucatu,BR,84.87,few clouds,-22.8858,-48.4450,Primar Plaza Hotel
18,Kudahuvadhoo,MV,82.47,few clouds,2.6708,72.8944,Niyama Private Islands Maldives
24,Butaritari,KI,82.62,scattered clouds,3.0707,172.7902,Isles Sunset Lodge
25,Karratha,AU,88.12,few clouds,-20.7377,116.8463,ibis Styles Karratha
31,Isangel,VU,77.59,overcast clouds,-19.5500,169.2667,Tanna Lodge
33,Mahebourg,MU,79.25,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa


In [16]:
clean_hotel_df.isnull().sum()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [17]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [22]:
# 11a. Add a marker layer for each city to the map. 
# 4a. Add a marker layer for each city to the map.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))